In [ ]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
dir_cmc_clim='F:/data/sst/cmc/CMC0.2deg/v2/climatology/'
def get_filename(lyr,idyjl):
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
    return cmc_filename
def get_filename_v3(lyr,idyjl):
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    dir_cmc='F:/data/sst/cmc/CMC0.1deg/v3/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    fname_tem=syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc'
    cmc_filename = dir_cmc + syr + '/' + sjdy + '/' + fname_tem
    return cmc_filename

In [ ]:
#make cmc monthly average ssts
for lyr in range(1992,2017): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        init = 0
        for idyjl in range(1,366):
            d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
            if d.month!=imon:
                continue
            cmc_filename = get_filename(lyr,idyjl)
            print(cmc_filename)
            ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])
            ds_masked = ds.where(ds['mask'] == 1.) 
            ds.close()
            ds_masked['sq_sst']=ds_masked.analysed_sst**2
            if init==0:
                ds_sum = ds_masked
                init = 1
            else:
                ds_sum = xr.concat([ds_sum,ds_masked],dim = 'time')
            print(idyjl,ds_sum.dims)
        ds_clim2 = ds_sum.resample(time='M').mean()    
#        ds_clim2 = ds_sum.groupby('time.month').mean('time')
        #ds_sum = ds_sum.mean('time',skipna=True)
        ds_mnth.append(ds_clim2)
    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='monthly/' + str(lyr) + 'monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    combined.to_netcdf(cmc_filename_out)
    

In [ ]:
#make cmc monthly average ssts FOR 0.1 deg data
#first read in 0.2 deg grid to interpolate onto

#testing 0.1 CMC
filename = 'F:/data/sst/cmc/CMC0.2deg/v2/1992/011/19920111120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])

for lyr in range(2017,2019): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        init = 0
        for idyjl in range(1,366):
            d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
            if d.month!=imon:
                continue
            cmc_filename = get_filename_v3(lyr,idyjl)
            print(cmc_filename)
            ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])
            ds_masked = ds.where(ds['mask'] == 1.) 
            ds_masked['sq_sst']=ds_masked.analysed_sst**2
            ds_interp = ds_masked.interp(lat = new_lat,lon = new_lon)            
            ds.close()
            if init==0:
                ds_sum = ds_interp
                init = 1
            else:
                ds_sum = xr.concat([ds_sum,ds_interp],dim = 'time')
            #print(idyjl,ds_sum.dims)
        ds_clim2 = ds_sum.resample(time='M').mean()    
#        ds_clim2 = ds_sum.groupby('time.month').mean('time')
        #ds_sum = ds_sum.mean('time',skipna=True)
        ds_mnth.append(ds_clim2)
    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='monthly/' + str(lyr) + 'monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    combined.to_netcdf(cmc_filename_out)
    

In [ ]:
#need to convert 2017 and 2018 to float 32 otherwise they don't combine with other index nicely
#testing data files 
file1='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2016monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2017monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds1=xr.open_dataset(file1)
ds2=xr.open_dataset(file2)
ds1.close()
ds2.close()
ds3 = ds2.astype('float32')
ds3['lon']=ds1.lon
ds3['lat']=ds1.lat
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2017monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds3.to_netcdf(file2)

#need to convert 2017 and 2018 to float 32 otherwise they don't combine with other index nicely
#testing data files 
file1='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2016monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2018monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds1=xr.open_dataset(file1)
ds2=xr.open_dataset(file2)
ds1.close()
ds2.close()
ds3 = ds2.astype('float32')
ds3['lon']=ds1.lon
ds3['lat']=ds1.lat
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2018monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds3.to_netcdf(file2)


In [ ]:
#testing data files 
file1='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2016monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
file2='F:/data/sst/cmc/CMC0.2deg/v2/monthly/2018monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds1=xr.open_dataset(file1)
ds2=xr.open_dataset(file2)
ds3 = ds2.astype('float32')
ds3['lon']=ds1.lon
ds3['lat']=ds1.lat
#print(ds3)
ds1.close()
ds2.close()
ds3 = xr.concat([ds3,ds1],dim = 'time')
print(ds3)

In [ ]:
#make cmc climatology from monthly files
for icase in range(0,3):
    if icase==0:
        iyr1,iyr2 = 1992,2000
    if icase==1:
        iyr1,iyr2 = 2000,2010
    if icase==2:
        iyr1,iyr2 = 2010,2018
    init = 0
    for lyr in range(iyr1,iyr2): 
        fname_tem='monthly/' + str(lyr) + 'monthly_average_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
        cmc_filename_out = dir_cmc + fname_tem
        ds = xr.open_dataset(cmc_filename_out)
        ds_masked = ds.where(ds['mask'] == 1.) 
        if init==0:
            ds_sum = ds_masked
            init = 1
        else:
            ds_sum = xr.concat([ds_sum,ds_masked],dim = 'time')
        print(lyr,ds_sum.dims)
    ds_sum2 = ds_sum.groupby('time.month').mean('time')
    fname_tem='monthly/monthly_climatology_'+str(iyr1)+'_'+str(iyr2-1)+'_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = dir_cmc + fname_tem
    ds_sum2.to_netcdf(cmc_filename_out)


In [ ]:
num_year = 2017-1992+1
num_year_file1 = 1999 - 1992 +1
num_year_file2 = 2009 - 2000 +1 
num_year_file3 = 2017 - 2010 +1
frac_file1 = num_year_file1 / num_year
frac_file2 = num_year_file2 / num_year
frac_file3 = num_year_file3 / num_year
fname_tem=dir_cmc + 'monthly/monthly_climatology_1992_1999_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked = ds.where(ds['mask'] == 1.) 
ds.close()
fname_tem=dir_cmc + 'monthly/monthly_climatology_2000_2009_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked2 = ds.where(ds['mask'] == 1.) 
#ds_sum = xr.concat([ds_masked2,ds_masked],dim = 'time')
ds.close()
fname_tem=dir_cmc + 'monthly/monthly_climatology_2010_2017_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds_masked3 = ds.where(ds['mask'] == 1.) 
#ds_sum = xr.concat([ds_masked2,ds_masked],dim = 'time')
ds.close()
ds_ave = frac_file1*ds_masked + frac_file2*ds_masked2 + frac_file3*ds_masked3
#ds_sum = ds_sum.mean('time',skipna=True)
print(frac_file1+frac_file2+frac_file3,frac_file1,frac_file2,frac_file3)
fname_tem=dir_cmc + 'monthly/monthly_climatology_1992_2017_120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
print(fname_tem)
ds_ave.to_netcdf(fname_tem)